In [1]:
import tensorflow as tf
from tensorflow.python.training.moving_averages import assign_moving_average
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes, count
n_classes = 50
count=0 

In [2]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=256):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn

def setup_inputs(sess, filenames, training_img_dir, image_size=256, crop_size=224, isTest=False, batch_size=64):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_hue(image,0.05)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        

    image = tf.random_crop(image, [crop_size, crop_size, 3])
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)


In [3]:
def activation(x,name="activation"):
    return tf.nn.swish(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, name="bn"+name)

def initializer(in_filters, out_filters, name):
    w1 = tf.get_variable(name+"W", [3, 3, in_filters, out_filters], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return w1, b1

def merge(in_x, growth_rate, isTraining, name):
    channels = int(in_x.get_shape()[-1])
    w1, b1 = initializer(channels, growth_rate, name+"merge_unit")
    x = batchnorm(in_x, isTraining, name=name+'mer')
    x = activation(x)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")
    return x

def block_unit(in_x, growth_rate,isTraining, name):
    channels = int(in_x.get_shape()[-1])
    w1, b1 = initializer(channels, 4*growth_rate, name+"block_unit")
    x = batchnorm(in_x, isTraining, name=name+'unit')
    x = activation(x)
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")
    x = merge(x, growth_rate, isTraining, name) #NOTE: output/input_tensor_depth is different from input
    return x
    
def transition_layer(in_x,isTraining, theta, name):
    channels = int(in_x.get_shape()[-1])
    x = batchnorm(in_x, isTraining, name=name+'tra')
    w1, b1 = initializer(channels, channels*theta, name+"transition_lay")
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")
    x = tf.nn.avg_pool(value=x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name+'avg_pool_2x2')
    return x

In [4]:

def dense_block(in_x, growth_rate,isTraining, name):
    channels = int(in_x.get_shape()[-1])
    with tf.name_scope("block1") as scope:
        x = block_unit(in_x, growth_rate, isTraining, name+"block1")#NOTE:64 is 2k, here k = 32, 128 is 4k, output is k = 32
        x = tf.concat(values=[in_x, x], axis=3, name=name+'stack0')# 96
        print("Denseblock1 with channels %d"%(x.get_shape().as_list()[-1]))

    with tf.name_scope("block2") as scope:
        out_x = block_unit(x, growth_rate, isTraining, name+"block2")
        x = tf.concat(values=[x, out_x], axis=3, name=name+'stack1')
        print("Denseblock2 with channels %d"%(x.get_shape().as_list()[-1]))

    with tf.name_scope("block3") as scope:
        out_x = block_unit(x, growth_rate, isTraining, name+"block3")
        x = tf.concat(values=[x, out_x], axis=3, name=name+'stack2')
        print("Denseblock3 with channels %d"%(x.get_shape().as_list()[-1]))

    with tf.name_scope("block4") as scope:
        out_x = block_unit(x, growth_rate, isTraining, name+"block4")
        x = tf.concat(values=[x, out_x], axis=3, name=name+'stack3')
        print("Denseblock4 with channels %d"%(x.get_shape().as_list()[-1]))

    return x

      
def DenseNet(_X, isTraining,growth_rate):
    global n_classes
    w1 = tf.get_variable("initW", [7, 7, 3, 96], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("initB", [96], initializer=tf.truncated_normal_initializer())
    x = conv2d('conv1', _X, w1, b1, 4, "VALID")
    num_blocks = 4
    for j in range(num_blocks):
        x =dense_block(x, growth_rate, isTraining, "denb%d"%(j))
        
        x =transition_layer(x, isTraining, 0.5, "transb%d"%(j))

    channels = int(x.get_shape()[-1])
    wo, bo=initializer(channels, n_classes, "FinalOutput")
    x = conv2d('final', x, wo, bo, 1, "SAME")
    
    x = activation(x)
    x = batchnorm(x, isTraining, name='FinalBn')
    
    x = tf.reduce_mean(x, [1,2])
    W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())
    
    out = tf.matmul(x, W) + b
                            

    return out

In [ ]:
batch_size = 32
display_step = 20
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-3                          # Learning rate start
print('GO!!')
with open('val.txt','r') as fp:
    tlen1 = len(fp.readlines())

GO!!


In [ ]:
# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "train.txt", "", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "val.txt", "",isTest=True, batch_size=batch_size)

max_iter = glen1*400
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("DenseNet") as scope:
  pred = DenseNet(train_data, True, 24)
  scope.reuse_variables()
  valpred = DenseNet(val_data, False, 24)

with tf.name_scope('Loss_and_Accuracy'):
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  with tf.control_dependencies(update_ops):
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  
  correct_prediction2 = tf.equal(tf.argmax(valpred,1), val_labels)
  accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
  
  tf.summary.scalar('Loss', cost)
  tf.summary.scalar('Training_Accuracy', accuracy)


Preparing the training & validation data...
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0
writer = tf.summary.FileWriter("log2", sess.graph)
summaries = tf.summary.merge_all()

print("We are going to train the ImageNet model based on denseNet!!!")
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-3) & (epoch1 >2)):
        lr /= 10

    sess.run(optimizer,  feed_dict={learning_rate: lr})

    if (step % 1000==1) & (step>1000):
        save_path = saver.save(sess, "./tf_densenet_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step>1 and step % display_step == 1:
        # calculate the loss
        loss, acc, summaries_string = sess.run([cost, accuracy, summaries])
        print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, acc,  lr))
        writer.add_summary(summaries_string, step)
        
    if step>1 and (step % (display_step*10) == 0):
        rounds = tlen1 // batch_size
        
        valacc=[]
        for k in range(rounds):
            acc= sess.run(accuracy2)
            valacc.append(acc)
        print("\nIter=%d/epoch=%d, Validation Accuracy=%.6f" % (step*batch_size, epoch1 , np.mean(valacc)))

  
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "./tfdensenet_model.ckpt")
print("Model saved in file: %s" % save_path)
sess.close()

We are going to train the ImageNet model based on denseNet!!!
Iter=672/epoch=0, Loss=7.807419, Training Accuracy=0.000000, lr=0.001000
Iter=1312/epoch=0, Loss=6.838737, Training Accuracy=0.000000, lr=0.001000
Iter=1952/epoch=0, Loss=5.792522, Training Accuracy=0.062500, lr=0.001000
Iter=2592/epoch=0, Loss=5.675241, Training Accuracy=0.000000, lr=0.001000
Iter=3232/epoch=0, Loss=5.670481, Training Accuracy=0.031250, lr=0.001000
Iter=3872/epoch=0, Loss=6.387673, Training Accuracy=0.031250, lr=0.001000
Iter=4512/epoch=0, Loss=6.495638, Training Accuracy=0.093750, lr=0.001000
Iter=5152/epoch=0, Loss=5.403708, Training Accuracy=0.031250, lr=0.001000
Iter=5792/epoch=0, Loss=6.241925, Training Accuracy=0.031250, lr=0.001000

Iter=6400/epoch=0, Validation Accuracy=0.024554
Iter=6432/epoch=0, Loss=4.965719, Training Accuracy=0.093750, lr=0.001000
Iter=7072/epoch=0, Loss=5.233444, Training Accuracy=0.000000, lr=0.001000
Iter=7712/epoch=0, Loss=5.229833, Training Accuracy=0.093750, lr=0.001000
It

Iter=64672/epoch=0, Loss=3.961715, Training Accuracy=0.062500, lr=0.001000
Iter=65312/epoch=0, Loss=4.295077, Training Accuracy=0.000000, lr=0.001000
Iter=65952/epoch=0, Loss=3.946056, Training Accuracy=0.093750, lr=0.001000
Iter=66592/epoch=0, Loss=4.034994, Training Accuracy=0.156250, lr=0.001000
Iter=67232/epoch=0, Loss=3.856010, Training Accuracy=0.031250, lr=0.001000
Iter=67872/epoch=0, Loss=3.950647, Training Accuracy=0.062500, lr=0.001000
Iter=68512/epoch=0, Loss=3.880178, Training Accuracy=0.125000, lr=0.001000
Iter=69152/epoch=0, Loss=4.138126, Training Accuracy=0.093750, lr=0.001000
Iter=69792/epoch=0, Loss=4.262732, Training Accuracy=0.125000, lr=0.001000

Iter=70400/epoch=0, Validation Accuracy=0.109375
Iter=70432/epoch=0, Loss=3.981267, Training Accuracy=0.156250, lr=0.001000
Iter=71072/epoch=0, Loss=3.637785, Training Accuracy=0.125000, lr=0.001000
Iter=71712/epoch=0, Loss=3.676419, Training Accuracy=0.062500, lr=0.001000
Iter=72352/epoch=0, Loss=4.160649, Training Accura

Iter=128672/epoch=1, Loss=3.258851, Training Accuracy=0.125000, lr=0.001000
Iter=129312/epoch=1, Loss=3.434563, Training Accuracy=0.125000, lr=0.001000
Iter=129952/epoch=1, Loss=3.269869, Training Accuracy=0.250000, lr=0.001000
Iter=130592/epoch=1, Loss=3.854667, Training Accuracy=0.156250, lr=0.001000
Iter=131232/epoch=1, Loss=3.422405, Training Accuracy=0.156250, lr=0.001000
Iter=131872/epoch=1, Loss=3.471973, Training Accuracy=0.125000, lr=0.001000
Iter=132512/epoch=1, Loss=3.101473, Training Accuracy=0.187500, lr=0.001000
Iter=133152/epoch=1, Loss=3.819543, Training Accuracy=0.187500, lr=0.001000
Iter=133792/epoch=1, Loss=3.627036, Training Accuracy=0.125000, lr=0.001000

Iter=134400/epoch=1, Validation Accuracy=0.092634
Iter=134432/epoch=1, Loss=3.840149, Training Accuracy=0.125000, lr=0.001000
Iter=135072/epoch=1, Loss=4.063134, Training Accuracy=0.031250, lr=0.001000
Iter=135712/epoch=1, Loss=3.370374, Training Accuracy=0.156250, lr=0.001000
Iter=136352/epoch=1, Loss=3.531100, T

Model saved in file at iteration 192032: ./tf_densenet_model_iter6001.ckpt
Iter=192032/epoch=1, Loss=3.845389, Training Accuracy=0.093750, lr=0.001000
Iter=192672/epoch=1, Loss=3.081644, Training Accuracy=0.250000, lr=0.001000
Iter=193312/epoch=1, Loss=3.295479, Training Accuracy=0.093750, lr=0.001000
Iter=193952/epoch=1, Loss=3.086167, Training Accuracy=0.187500, lr=0.001000
Iter=194592/epoch=1, Loss=3.245038, Training Accuracy=0.093750, lr=0.001000
Iter=195232/epoch=1, Loss=3.329552, Training Accuracy=0.312500, lr=0.001000
Iter=195872/epoch=1, Loss=2.729349, Training Accuracy=0.312500, lr=0.001000
Iter=196512/epoch=1, Loss=3.110675, Training Accuracy=0.187500, lr=0.001000
Iter=197152/epoch=1, Loss=3.773790, Training Accuracy=0.218750, lr=0.001000
Iter=197792/epoch=1, Loss=3.745154, Training Accuracy=0.156250, lr=0.001000

Iter=198400/epoch=1, Validation Accuracy=0.181920
Iter=198432/epoch=1, Loss=3.074841, Training Accuracy=0.187500, lr=0.001000
Iter=199072/epoch=1, Loss=3.189715, Tr


Iter=256000/epoch=2, Validation Accuracy=0.239955
Model saved in file at iteration 256032: ./tf_densenet_model_iter8001.ckpt
Iter=256032/epoch=2, Loss=2.712805, Training Accuracy=0.250000, lr=0.001000
Iter=256672/epoch=2, Loss=3.619224, Training Accuracy=0.187500, lr=0.001000
Iter=257312/epoch=2, Loss=3.186782, Training Accuracy=0.218750, lr=0.001000
Iter=257952/epoch=2, Loss=3.537404, Training Accuracy=0.187500, lr=0.001000
Iter=258592/epoch=2, Loss=2.809527, Training Accuracy=0.250000, lr=0.001000
Iter=259232/epoch=2, Loss=3.210867, Training Accuracy=0.125000, lr=0.001000
Iter=259872/epoch=2, Loss=3.202291, Training Accuracy=0.156250, lr=0.001000
Iter=260512/epoch=2, Loss=2.646619, Training Accuracy=0.250000, lr=0.001000
Iter=261152/epoch=2, Loss=3.260927, Training Accuracy=0.312500, lr=0.001000
Iter=261792/epoch=2, Loss=3.285800, Training Accuracy=0.125000, lr=0.001000

Iter=262400/epoch=2, Validation Accuracy=0.237723
Iter=262432/epoch=2, Loss=3.209051, Training Accuracy=0.187500,

...

Iter=1584672/epoch=12, Loss=2.452800, Training Accuracy=0.375000, lr=0.000100
Iter=1585312/epoch=12, Loss=2.052290, Training Accuracy=0.437500, lr=0.000100
Iter=1585952/epoch=12, Loss=2.288027, Training Accuracy=0.406250, lr=0.000100
Iter=1586592/epoch=12, Loss=2.147201, Training Accuracy=0.500000, lr=0.000100

Iter=1587200/epoch=12, Validation Accuracy=0.436384
Iter=1587232/epoch=12, Loss=2.228517, Training Accuracy=0.375000, lr=0.000100
Iter=1587872/epoch=12, Loss=1.985860, Training Accuracy=0.343750, lr=0.000100
Iter=1588512/epoch=12, Loss=2.669905, Training Accuracy=0.375000, lr=0.000100
Iter=1589152/epoch=12, Loss=2.385065, Training Accuracy=0.406250, lr=0.000100
Iter=1589792/epoch=12, Loss=2.021894, Training Accuracy=0.500000, lr=0.000100
Iter=1590432/epoch=12, Loss=1.729329, Training Accuracy=0.562500, lr=0.000100
Iter=1591072/epoch=12, Loss=2.130864, Training Accuracy=0.500000, lr=0.000100
Iter=1591712/epoch=12, Loss=2.132607, Training Accuracy=0.437500, lr=0.000100
Iter=159235

Iter=1646752/epoch=12, Loss=2.439831, Training Accuracy=0.375000, lr=0.000100
Iter=1647392/epoch=12, Loss=2.184305, Training Accuracy=0.531250, lr=0.000100
Iter=1648032/epoch=12, Loss=2.585201, Training Accuracy=0.437500, lr=0.000100
Iter=1648672/epoch=12, Loss=2.384521, Training Accuracy=0.375000, lr=0.000100
Iter=1649312/epoch=12, Loss=1.937416, Training Accuracy=0.656250, lr=0.000100
Iter=1649952/epoch=12, Loss=2.343494, Training Accuracy=0.406250, lr=0.000100
Iter=1650592/epoch=12, Loss=2.188044, Training Accuracy=0.531250, lr=0.000100

Iter=1651200/epoch=12, Validation Accuracy=0.431920
Iter=1651232/epoch=12, Loss=2.297215, Training Accuracy=0.406250, lr=0.000100
Iter=1651872/epoch=12, Loss=2.071112, Training Accuracy=0.406250, lr=0.000100
Iter=1652512/epoch=12, Loss=1.656175, Training Accuracy=0.562500, lr=0.000100
Iter=1653152/epoch=12, Loss=1.962408, Training Accuracy=0.437500, lr=0.000100
Iter=1653792/epoch=12, Loss=1.882871, Training Accuracy=0.468750, lr=0.000100
Iter=165443


Iter=1708800/epoch=13, Validation Accuracy=0.450893
Iter=1708832/epoch=13, Loss=2.437619, Training Accuracy=0.343750, lr=0.000100
Iter=1709472/epoch=13, Loss=2.164026, Training Accuracy=0.343750, lr=0.000100
Iter=1710112/epoch=13, Loss=1.963714, Training Accuracy=0.500000, lr=0.000100
Iter=1710752/epoch=13, Loss=1.679650, Training Accuracy=0.437500, lr=0.000100
Iter=1711392/epoch=13, Loss=1.923410, Training Accuracy=0.437500, lr=0.000100
Iter=1712032/epoch=13, Loss=3.049421, Training Accuracy=0.281250, lr=0.000100
Iter=1712672/epoch=13, Loss=1.825582, Training Accuracy=0.500000, lr=0.000100
Iter=1713312/epoch=13, Loss=2.275497, Training Accuracy=0.343750, lr=0.000100
Iter=1713952/epoch=13, Loss=2.023242, Training Accuracy=0.468750, lr=0.000100
Iter=1714592/epoch=13, Loss=2.571919, Training Accuracy=0.343750, lr=0.000100

Iter=1715200/epoch=13, Validation Accuracy=0.426339
Iter=1715232/epoch=13, Loss=2.357147, Training Accuracy=0.406250, lr=0.000100
Iter=1715872/epoch=13, Loss=2.428443

Iter=1770912/epoch=13, Loss=1.713547, Training Accuracy=0.500000, lr=0.000100
Iter=1771552/epoch=13, Loss=2.297004, Training Accuracy=0.343750, lr=0.000100
Iter=1772192/epoch=13, Loss=2.329874, Training Accuracy=0.375000, lr=0.000100

Iter=1772800/epoch=13, Validation Accuracy=0.435268
Iter=1772832/epoch=13, Loss=2.274540, Training Accuracy=0.281250, lr=0.000100
Iter=1773472/epoch=13, Loss=2.488975, Training Accuracy=0.375000, lr=0.000100
Iter=1774112/epoch=13, Loss=2.510805, Training Accuracy=0.312500, lr=0.000100
Iter=1774752/epoch=13, Loss=1.973266, Training Accuracy=0.375000, lr=0.000100
Iter=1775392/epoch=13, Loss=2.258759, Training Accuracy=0.437500, lr=0.000100
Iter=1776032/epoch=13, Loss=2.057205, Training Accuracy=0.468750, lr=0.000100
Iter=1776672/epoch=13, Loss=2.490748, Training Accuracy=0.281250, lr=0.000100
Iter=1777312/epoch=13, Loss=2.324169, Training Accuracy=0.375000, lr=0.000100
Iter=1777952/epoch=13, Loss=2.321272, Training Accuracy=0.343750, lr=0.000100
Iter=177859

Iter=1832992/epoch=14, Loss=1.812457, Training Accuracy=0.468750, lr=0.000100
Iter=1833632/epoch=14, Loss=2.419190, Training Accuracy=0.375000, lr=0.000100
Iter=1834272/epoch=14, Loss=2.134389, Training Accuracy=0.437500, lr=0.000100
Iter=1834912/epoch=14, Loss=1.854194, Training Accuracy=0.500000, lr=0.000100
Iter=1835552/epoch=14, Loss=2.357246, Training Accuracy=0.343750, lr=0.000100
Iter=1836192/epoch=14, Loss=2.188159, Training Accuracy=0.468750, lr=0.000100

Iter=1836800/epoch=14, Validation Accuracy=0.439732
Iter=1836832/epoch=14, Loss=1.800017, Training Accuracy=0.468750, lr=0.000100
Iter=1837472/epoch=14, Loss=2.186322, Training Accuracy=0.312500, lr=0.000100
Iter=1838112/epoch=14, Loss=1.914082, Training Accuracy=0.468750, lr=0.000100
Iter=1838752/epoch=14, Loss=1.648366, Training Accuracy=0.562500, lr=0.000100
Iter=1839392/epoch=14, Loss=2.076680, Training Accuracy=0.437500, lr=0.000100
Iter=1840032/epoch=14, Loss=2.097458, Training Accuracy=0.468750, lr=0.000100
Iter=184067

In [ ]:
exit()